In [1]:
import torch
import numpy as np
torch.cuda.is_available()

c:\Users\Anubhav\anaconda3\envs\mytorch112\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
import sys
print(sys.version)

3.8.0 (default, Nov  6 2019, 16:00:02) [MSC v.1916 64 bit (AMD64)]


In [3]:
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import datetime
import os
import torchvision
import time
import copy
# from torch.utils.tensorboard import SummaryWriter

# from torchsummary import summary

from tqdm import tqdm

In [4]:
torch.__version__

'1.12.1+cu116'

In [5]:
# misc functions (https://github.com/choasma/HSIC-bottleneck/blob/master/source/hsicbt/utils/misc.py)

def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return torch.squeeze(torch.eye(num_classes)[y])

def get_layer_parameters(model, idx_range):

    param_out = []
    param_out_name = []
    for it, (name, param) in enumerate(model.named_parameters()):
        if it in idx_range:
            param_out.append(param)
            param_out_name.append(name)

    return param_out, param_out_name

# https://github.com/choasma/HSIC-bottleneck/blob/master/source/hsicbt/utils/meter.py
class AverageMeter(object):
    """Basic meter"""
    def __init__(self):
        self.reset()

    def reset(self):
        """ reset meter
        """
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        """ incremental meter
        """
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def get_accuracy_hsic(model, dataloader):
    """ Computes the precision@k for the specified values of k
        https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """
    output_list = []
    target_list = []
    for batch_idx, (data, target) in enumerate(dataloader):
        output, hiddens = model(data.to(next(model.parameters()).device))
        output = output.cpu().detach().numpy()
        target = target.cpu().detach().numpy().reshape(-1,1)
        output_list.append(output)
        target_list.append(target)
    output_arr = np.vstack(output_list)
    target_arr = np.vstack(target_list)
    avg_acc = 0
    reorder_list = []
    for i in range(10):
        indices = np.where(target_arr==i)[0]
        select_item = output_arr[indices]
        out = np.array([np.argmax(vec) for vec in select_item])
        y = np.mean(select_item, axis=0)
        while np.argmax(y) in reorder_list:
            y[np.argmax(y)] = 0
        reorder_list.append(np.argmax(y))
        num_correct = np.where(out==np.argmax(y))[0]
        accuracy = float(num_correct.shape[0])/float(out.shape[0])
        avg_acc += accuracy
    avg_acc /= 10.

    return avg_acc*100., reorder_list


def get_accuracy(output, target, topk=(1,)):
    """ Computes the precision@k for the specified values of k
        https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """
    maxk = max(topk)
    batch_size = target.size(0)
    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))
    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

def get_accuracy_epoch(model, dataloader):
    """ Computes the precision@k for the specified values of k
        https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """
    output_list = []
    target_list = []
    acc = []
    loss = []
    cross_entropy_loss = torch.nn.CrossEntropyLoss()
    model = model.to('cuda')
    device = next(model.parameters()).device

    for batch_idx, (data, target) in enumerate(dataloader):
        data = data.to(device)
        target = target.to(device)
        output, hiddens = model(data)
        loss.append(cross_entropy_loss(output, target).cpu().detach().numpy())
        acc.append(get_accuracy(output, target)[0].cpu().detach().numpy())
    return np.mean(acc), np.mean(loss)


In [6]:
# Implement Gausian kernel function to calculate K_X and K_y
# gausian kernel, k(x, y) ~ exp(-(1/2)*||x - y||^2/sigma**2 )

def distmat(X):
    """ distance matrix
        Euclidean Distance Matrix (EDM)
        D = abs(a^2 + b^2 - 2a.b_T)
    """
    r = torch.sum(X*X, 1)
    r = r.view([-1, 1])
    a = torch.mm(X, torch.transpose(X,0,1))
    D = r.expand_as(a) - 2*a +  torch.transpose(r,0,1).expand_as(a)
    D = torch.abs(D)

    return D

def kernelmat(X, sigma):
    """
    Kernel function

    m: training batch size
    H: centering matrix:: I_m - (1/m)*1_m.1_m
    gausian kernel: k(x, y) ~ exp(-(1/2)*||x - y||^2/sigma**2)
    """
    m = int(X.size()[0]) # batch size
    H = torch.eye(m) - (1./m) * torch.ones([m,m])

    Dxx = distmat(X)

    variance = 2.*sigma*sigma*X.size()[1]            
    Kx = torch.exp(-Dxx / variance).type(torch.FloatTensor)   # kernel
    Kxc = torch.mm(Kx, H) # kernel function centered with H

    return Kxc


def hsic_base(x, y, sigma=None, use_cuda=True):
    """
    Implement equation 3 in the paper
    HSIC: (m - 1)^-2 . trace(Kx H Ky H)
    """
    m = int(x.size()[0]) # batch size

    KxH = kernelmat(x, sigma=sigma)
    KyH = kernelmat(y, sigma=sigma)

    return torch.trace(KxH @ KyH)/(m - 1)**2

# taken from HSIC implementation 
# https://github.com/choasma/HSIC-bottleneck/blob/9f1fe2447592d61c0ba524aad0ff0820ae2ba9cb/source/hsicbt/core/train_misc.py#L26
# def hsic_objective(hidden, h_target, h_data, sigma):

#     hsic_hy_val = hsic_base( hidden, h_target, sigma=sigma)
#     hsic_hx_val = hsic_base( hidden, h_data,   sigma=sigma)

#     return hsic_hx_val, hsic_hy_val

def hsic_loss_obj(hidden, h_target, h_data, sigma):
    """
    calculate hsic between input (X) and hidden layer weights
    calculate hsic between hidden layer weights and target (Y)

    return: hx, hy for calculating loss in training pipeline
    """
    hsic_hx = hsic_base(hidden, h_data, sigma=sigma)
    hsic_hy = hsic_base(hidden, h_target, sigma=sigma)

    return hsic_hx, hsic_hy

In [7]:
from torchvision.datasets import CIFAR10, MNIST
from torch.utils.data import DataLoader

In [8]:
# prepare data loader for CIFAR10 and MNIST

train_transform = transforms.Compose([transforms.ToTensor()]) # , transforms.Resize(size=(227, 227))
valid_transform = train_transform

train_set = CIFAR10('./data/cifar10', train=True,
                  download=True, transform=train_transform)
valid_set = CIFAR10('./data/cifar10', train=False,
                  download=True, transform=valid_transform)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True)
val_loader = torch.utils.data.DataLoader(valid_set, batch_size=128, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
# create primitive conv block with conv2d, bn, and activation.

def get_activation(atype):

    if atype=='relu':
        nonlinear = nn.ReLU()
    elif atype=='tanh':
        nonlinear = nn.Tanh() 
    elif atype=='sigmoid':
        nonlinear = nn.Sigmoid() 
    elif atype=='elu':
        nonlinear = nn.ELU()

    return nonlinear

def makeblock_conv(in_chs, out_chs, atype, stride=1):

    layer = nn.Conv2d(in_channels=in_chs, 
        out_channels=out_chs, kernel_size=5, stride=stride)
    bn = nn.BatchNorm2d(out_chs, affine=False)
    nonlinear = get_activation(atype)

    return nn.Sequential(*[layer, bn, nonlinear])

def makeblock_dense(in_dim, out_dim, atype):
    
    layer = nn.Linear(in_dim, out_dim)
    bn = nn.BatchNorm1d(out_dim, affine=False)
    nonlinear = get_activation(atype)
    out = nn.Sequential(*[layer, bn, nonlinear])
    
    return out

In [38]:
class ModelConv(nn.Module):

    def __init__(self, in_width=784, hidden_width=64, n_layers=5, atype='relu', 
        last_hidden_width=None, data_code='cifar10', **kwargs):
        super(ModelConv, self).__init__()
    
        block_list = []
        is_conv = False

        if data_code == 'cifar10':
            in_ch = 3
        elif data_code == 'mnist':
            in_ch = 1

        last_hw = hidden_width
        if last_hidden_width:
            last_hw = last_hidden_width
        
        for i in range(n_layers):
            block = makeblock_conv(hidden_width, hidden_width, atype)
            block_list.append(block)

        self.input_layer    = makeblock_conv(in_ch, hidden_width, atype)
        self.sequence_layer = nn.Sequential(*block_list)
        if data_code == 'mnist':
            dim = 2048
        elif data_code == 'cifar10':
            dim = 8192

        self.output_layer = makeblock_dense(dim, last_hw, atype)

        self.is_conv = is_conv
        self.in_width = in_width

    def forward(self, x):

        output_list = []
        
        x = self.input_layer(x)
        output_list.append(x)
        
        for block in self.sequence_layer:
            x = block(x.clone())
            output_list.append(x)
            
        x = x.view(-1, np.prod(x.size()[1:]))

        # print(x.size())

        x = self.output_layer(x)
        output_list.append(x)

        return x, output_list

In [11]:
class ModelVanilla(nn.Module):

    def __init__(self, hidden_width=64, last_hidden_width=None, **kwargs):
        super(ModelVanilla, self).__init__()
        last_dim = hidden_width
        if last_hidden_width:
        	last_dim = last_hidden_width
            
        self.output = nn.Linear(last_dim, 10)

    def forward(self, x):
        x = self.output(x)
        return F.log_softmax(x, dim=1)


In [12]:
class ModelEnsemble(nn.Module):

    def __init__(self, hsic_model, vanilla_model):
        super(ModelEnsemble, self).__init__()

        self._hsic_model = hsic_model
        self._vanilla_model = vanilla_model
        
    def forward(self, x):
        
        x, hiddens = self._hsic_model(x)
        x = self._vanilla_model(x)
        return x, hiddens

In [13]:
def standard_train(cepoch, model, data_loader, optimizer, config_dict):

    batch_acc    = AverageMeter()
    batch_loss   = AverageMeter()
    batch_hischx = AverageMeter()
    batch_hischy = AverageMeter()

    cross_entropy_loss = torch.nn.CrossEntropyLoss()
    prec1 = total_loss = hx_l = hy_l = -1

    batch_log = {}
    batch_log['batch_acc'] = []
    batch_log['batch_loss'] = []
    batch_log['batch_hsic_hx'] = []
    batch_log['batch_hsic_hy'] = []

    model = model.to(config_dict['device'])

    n_data = config_dict['batch_size'] * len(data_loader)
    
    pbar = tqdm(enumerate(data_loader), total=n_data/config_dict['batch_size'], ncols=150)

    for batch_idx, (data, target) in pbar:

        data   = data.to(config_dict['device'])
        target = target.to(config_dict['device'])
        output, hiddens = model(data)

        h_target = target.view(-1,1)
        h_target = to_categorical(h_target, num_classes=10).float()
        
        h_data = data.view(-1, np.prod(data.size()[1:]))

        optimizer.zero_grad()
        loss = cross_entropy_loss(output, target)
        loss.backward()
        optimizer.step()


        loss = float(loss.detach().cpu().numpy())
        prec1, prec5 = get_accuracy(output, target, topk=(1, 5)) 
        prec1 = float(prec1.cpu().numpy())
    
        batch_acc.update(prec1)   
        batch_loss.update(loss)  
        batch_hischx.update(hx_l)
        batch_hischy.update(hy_l)

        msg = 'Train Epoch: {cepoch} [ {cidx:5d}/{tolidx:5d} ({perc:2d}%)] Loss:{loss:.4f} Acc:{acc:.4f} hsic_xz:{hsic_zx:.4f} hsic_yz:{hsic_zy:.4f}'.format(
                        cepoch = cepoch,  
                        cidx = (batch_idx+1)*config_dict['batch_size'], 
                        tolidx = n_data,
                        perc = int(100. * (batch_idx+1)*config_dict['batch_size']/n_data), 
                        loss = batch_loss.avg, 
                        acc  = batch_acc.avg,
                        hsic_zx = batch_hischx.avg,
                        hsic_zy = batch_hischy.avg,
                    )

        # # # preparation log information and print progress # # #
        if ((batch_idx) % config_dict['log_batch_interval'] == 0): 
            batch_log['batch_acc'].append(batch_acc.val)
            batch_log['batch_loss'].append(batch_loss.val)
            batch_log['batch_hsic_hx'].append(batch_hischx.val)
            batch_log['batch_hsic_hy'].append(batch_hischy.val)

        pbar.set_description(msg)

    return batch_log

In [14]:
def hsic_train(cepoch, model, data_loader, config_dict):

    # cross_entropy_loss = torch.nn.CrossEntropyLoss()
    prec1 = total_loss = hx_l = hy_l = -1

    batch_acc    = AverageMeter()
    batch_loss   = AverageMeter()
    batch_hischx = AverageMeter()
    batch_hischy = AverageMeter()

    batch_log = {}
    batch_log['batch_acc'] = []
    batch_log['batch_loss'] = []
    batch_log['batch_hsic_hx'] = []
    batch_log['batch_hsic_hy'] = []

    model = model.to(config_dict['device'])

    n_data = config_dict['batch_size'] * len(data_loader)

    pbar = tqdm(enumerate(data_loader), total=n_data/config_dict['batch_size'], ncols=120)
    for batch_idx, (data, target) in pbar:

        data   = data.to(config_dict['device'])
        target = target.to(config_dict['device'])
        output, hiddens = model(data)

        h_target = target.view(-1,1)
        h_target = to_categorical(h_target, num_classes=10).float()
        h_data = data.view(-1, np.prod(data.size()[1:]))

        idx_range = []
        it = 0

        # So the batchnorm is not learnable, making only @,b at layer
        for i in range(len(hiddens)):
            idx_range.append(np.arange(it, it+2).tolist())
            it += 2
    
        for i in range(len(hiddens)):
            
            output, hiddens = model(data)
            params, param_names = get_layer_parameters(model=model, idx_range=idx_range[i]) # so we only optimize one layer at a time
            optimizer = optim.SGD(params, lr = config_dict['learning_rate'], momentum=.9, weight_decay=0.001)
            optimizer.zero_grad()
            
            if len(hiddens[i].size()) > 2:
                hiddens[i] = hiddens[i].view(-1, np.prod(hiddens[i].size()[1:]))

            hx_l, hy_l = hsic_loss_obj(
                    hiddens[i],
                    h_target=h_target.float(),
                    h_data=h_data,
                    sigma=config_dict['sigma'],
            )
            #print(torch.max(hiddens[i]).cpu().detach().numpy(), torch.min(hiddens[i]).cpu().detach().numpy(), torch.std(hiddens[i]).cpu().detach().numpy())
            loss = (hx_l - config_dict['lambda_y']*hy_l)
            if i == 0:
                loss.backward()
            else:
                loss.backward()
            optimizer.step()
            # sigma_optimizer.step()
        # if config_dict['hsic_solve']:
        # prec1, reorder_list = get_accuracy_hsic(model, data_loader)
        batch_acc.update(prec1)
        batch_loss.update(total_loss)
        batch_hischx.update(hx_l.cpu().detach().numpy())
        batch_hischy.update(hy_l.cpu().detach().numpy())

        # print('H_hx:{H_hx:.8f} H_hy:{H_hy:.8f}'.format(H_hx = hx_l, H_hy = hy_l))

        # # # preparation log information and print progress # # #

        msg = 'Train Epoch: {cepoch} [ {cidx:5d}/{tolidx:5d} ({perc:2d}%)] H_hx:{H_hx:.8f} H_hy:{H_hy:.8f}'.format(
                        cepoch = cepoch,  
                        cidx = (batch_idx+1)*config_dict['batch_size'], 
                        tolidx = n_data,
                        perc = int(100. * (batch_idx+1)*config_dict['batch_size']/n_data), 
                        H_hx = batch_hischx.avg, 
                        H_hy = batch_hischy.avg,
                )

        if ((batch_idx+1) % config_dict['log_batch_interval'] == 0):

            batch_log['batch_acc'].append(batch_acc.avg)
            batch_log['batch_loss'].append(batch_loss.avg)
            batch_log['batch_hsic_hx'].append(batch_hischx.avg)
            batch_log['batch_hsic_hy'].append(batch_hischy.avg)

        pbar.set_description(msg)

    return batch_log, model

In [15]:
# train_transform = transforms.Compose([transforms.ToTensor()]) # , transforms.Resize(size=(227, 227))
# valid_transform = train_transform

# train_set = MNIST('./data/mnist', train=True,
#                   download=True, transform=train_transform)
# valid_set = MNIST('./data/mnist', train=False,
#                   download=True, transform=valid_transform)

# train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True)
# val_loader = torch.utils.data.DataLoader(valid_set, batch_size=128, shuffle=False)

In [31]:
def load_mnist(dataFolderPath='./data/mnist', train=True, download=True, batchSize=64):
    
    train_transform = transforms.Compose([transforms.ToTensor()]) # , transforms.Resize(size=(227, 227))
    valid_transform = train_transform

    train_set = MNIST(dataFolderPath, train=train,
                  download=download, transform=train_transform)
    valid_set = MNIST(dataFolderPath, train=False,
                  download=True, transform=valid_transform)

    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batchSize, shuffle=True)
    val_loader = torch.utils.data.DataLoader(valid_set, batch_size=batchSize, shuffle=False)    

    return train_loader, val_loader


def load_cifar10(dataFolderPath='./data/cifar10', train=True, download=True, batchSize=64):
    
    train_transform = transforms.Compose([transforms.ToTensor()]) # , transforms.Resize(size=(227, 227))
    valid_transform = train_transform

    train_set = CIFAR10(dataFolderPath, train=train,
                  download=download, transform=train_transform)
    valid_set = CIFAR10(dataFolderPath, train=False,
                  download=True, transform=valid_transform)

    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batchSize, shuffle=True)
    val_loader = torch.utils.data.DataLoader(valid_set, batch_size=batchSize, shuffle=False)    

    return train_loader, val_loader

def get_data(data_name, batch_size):

    if data_name=='cifar10':
        dataPath = './data/cifar10'
        train_loader, test_loader=load_cifar10(dataPath, batchSize=batch_size)

    elif data_name=='mnist':
        dataPath = './data/mnist'
        train_loader, test_loader=load_mnist(dataPath, batchSize=batch_size)

    return train_loader, test_loader

In [17]:
# config_dict = {}
# config_dict['batch_size'] = 128
# config_dict['learning_rate'] = 0.001
# config_dict['lambda_y'] = 50.
# config_dict['sigma'] = 2.
# config_dict['task'] = 'hsic-train'
# config_dict['device'] = 'cuda'
# config_dict['log_batch_interval'] = 10

# # # # data prepreation
# # train_loader, test_loader = get_dataset_from_code('mnist', 128)

# # # # simple fully-connected model
# model = ModelConv(hidden_width=32,
#                     n_layers=5,
#                     atype='relu',
#                     last_hidden_width=None,
#                     data_code='mnist')

# # # # start to train
# epochs = 5
# for cepoch in range(epochs):
#     # you can also re-write hsic_train function
#     batch_log, model = hsic_train(cepoch, model, train_loader, config_dict)
#     # print(get_accuracy_epoch(model, train_loader))

In [18]:
from color import print_emph, print_highlight
from utils import model_save
import datetime

# TTYPE_STANDARD = 'backprop'
TTYPE_HSICTRAIN = 'hsictrain'
# TTYPE_FORMAT = 'format'
# TTYPE_UNFORMAT = 'unformat'

# 1. train the HSIC model with last hidden demensions != 10 and save the model.T_destination
## 2. Create ensemble model with hsic model + linear model with softmax and train for 10 epochs
## without backprop (only update last layer params by SGD optim.)

def training_hsic(config_dict):

    print_emph("HSIC-Bottleneck training")

    train_loader, test_loader = get_data(
        config_dict['data_code'], config_dict['batch_size'])
    #torch.manual_seed(config_dict['seed'])
    # model = model_distribution(config_dict)

    model = ModelConv(**config_dict)
    nepoch = config_dict['epochs']
    epoch_range = range(1, nepoch+1)

    batch_log_list = []
    epoch_log_dict = {}
    epoch_log_dict['train_acc'] = []
    epoch_log_dict['test_acc'] = []

    time_stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    for cepoch in epoch_range:

        log = hsic_train(cepoch, model, train_loader, config_dict)

        batch_log_list.append(log)

        # save with each indexed
        main_path = 'X:/Data Files/Huawei/'
        filename = os.path.join(main_path, config_dict['data_code'])
        model_report, model_data = model_save.create_dirs(filename, time_stamp)

        model_save_path = os.path.join(model_data, "model---{:04d}.pt".format(cepoch))
        # model_save_path = "{}---{:04d}.pt".format(model_data, cepoch)
        torch.save(model.state_dict(), model_save_path)

        log_dict = {}
        log_dict['batch_log_list'] = batch_log_list
        log_dict['epoch_log_dict'] = epoch_log_dict
        log_dict['config_dict'] = config_dict

        model_report_path = os.path.join(model_report, "log---{:04d}.npy".format(cepoch))
        # model_report_path = "{}.npy".format(model_report)
        np.save(model_report_path, log_dict)

    return batch_log_list, epoch_log_dict

In [19]:
config_dict = {}
config_dict['batch_size'] = 128
config_dict['learning_rate'] = 0.001
config_dict['n_layers'] = 5
config_dict['hidden_width'] = 128
config_dict['epochs'] = 15

config_dict['seed'] = 1234
config_dict['device'] = 'cuda'
config_dict['atype'] = 'relu'
config_dict['data_code'] = 'mnist'

config_dict['lambda_y'] = 100.
config_dict['sigma'] = 5.
config_dict['task'] = 'hsic-train'
config_dict['log_batch_interval'] = 10
config_dict['ext'] = 'pdf'

In [22]:
batch_log_list, epoch_log_dict = training_hsic(config_dict)

# # # # # # # # # # # # # # # # # # # #
#     HSIC-Bottleneck training
# # # # # # # # # # # # # # # # # # # #


Train Epoch: 1 [ 60032/60032 (100%)] H_hx:0.00000046 H_hy:0.00000061: 100%|█████████| 469/469.0 [01:32<00:00,  5.09it/s]


Model files saved in:  X:/Data Files/Huawei/mnist\exp_20221101-230528
Model report files saved in :  X:/Data Files/Huawei/mnist\exp_20221101-230528\reports
Model saved in :  X:/Data Files/Huawei/mnist\exp_20221101-230528\reports\model


Train Epoch: 2 [ 60032/60032 (100%)] H_hx:0.00000046 H_hy:0.00000061: 100%|█████████| 469/469.0 [01:40<00:00,  4.68it/s]


Model files saved in:  X:/Data Files/Huawei/mnist\exp_20221101-230528
Model report files saved in :  X:/Data Files/Huawei/mnist\exp_20221101-230528\reports
Model saved in :  X:/Data Files/Huawei/mnist\exp_20221101-230528\reports\model


...

Model files saved in:  X:/Data Files/Huawei/mnist\exp_20221101-230528
Model report files saved in :  X:/Data Files/Huawei/mnist\exp_20221101-230528\reports
Model saved in :  X:/Data Files/Huawei/mnist\exp_20221101-230528\reports\model


Train Epoch: 14 [ 60032/60032 (100%)] H_hx:0.00000046 H_hy:0.00000062: 100%|████████| 469/469.0 [01:39<00:00,  4.69it/s]


Model files saved in:  X:/Data Files/Huawei/mnist\exp_20221101-230528
Model report files saved in :  X:/Data Files/Huawei/mnist\exp_20221101-230528\reports
Model saved in :  X:/Data Files/Huawei/mnist\exp_20221101-230528\reports\model


Train Epoch: 15 [ 60032/60032 (100%)] H_hx:0.00000046 H_hy:0.00000062: 100%|████████| 469/469.0 [01:39<00:00,  4.71it/s]


Model files saved in:  X:/Data Files/Huawei/mnist\exp_20221101-230528
Model report files saved in :  X:/Data Files/Huawei/mnist\exp_20221101-230528\reports
Model saved in :  X:/Data Files/Huawei/mnist\exp_20221101-230528\reports\model


In [23]:
def training_format(config_dict, model_load_path):

    print_emph("Format training")

    train_loader, test_loader = get_data(
        config_dict['data_code'], config_dict['batch_size'])
    torch.manual_seed(config_dict['seed'])

    vanilla_model = ModelVanilla(**config_dict) # Implement this - Done
    torch.manual_seed(config_dict['seed'])

    hsic_model = ModelConv(**config_dict)

    optimizer = optim.SGD(filter(lambda p: p.requires_grad, vanilla_model.parameters()),
            lr = config_dict['learning_rate'], weight_decay=0.001)

    model = torch.load(model_load_path)

    hsic_model.load_state_dict(model)
    hsic_model.eval()

    ensemble_model = ModelEnsemble(hsic_model, vanilla_model) ## Implement this - Done

    batch_log_list = []
    epoch_log_dict = {}
    epoch_log_dict['train_acc'] = []
    epoch_log_dict['train_loss'] = []
    epoch_log_dict['test_acc'] = []
    epoch_log_dict['test_loss'] = []

    nepoch = config_dict['epochs']

    # WIP: test acc for untrained net
    test_acc, test_loss = get_accuracy_epoch(ensemble_model, test_loader)
    epoch_log_dict['test_acc'].append(test_acc)
    epoch_log_dict['test_loss'].append(test_loss)

    train_acc, train_loss = get_accuracy_epoch(ensemble_model, train_loader)
    epoch_log_dict['train_acc'].append(train_acc)
    epoch_log_dict['train_loss'].append(train_loss)

    time_stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    for cepoch in range(1, nepoch+1):
        log = standard_train(cepoch, ensemble_model, train_loader, optimizer, config_dict)
        batch_log_list.append(log)

        train_acc, train_loss = get_accuracy_epoch(ensemble_model, train_loader)
        epoch_log_dict['train_acc'].append(train_acc)
        epoch_log_dict['train_loss'].append(train_loss)
        test_acc, test_loss = get_accuracy_epoch(ensemble_model, test_loader)
        epoch_log_dict['test_acc'].append(test_acc)
        epoch_log_dict['test_loss'].append(test_loss)
        print_highlight("Epoch - [{:04d}]: Training Acc: {:.2f}".format(cepoch, train_acc), 'green')
        print_highlight("Epoch - [{:04d}]: Testing  Acc: {:.2f}".format(cepoch, test_acc), 'green')


        # log_dict = {}
        # log_dict['batch_log_list'] = batch_log_list
        # log_dict['epoch_log_dict'] = epoch_log_dict
        # log_dict['config_dict'] = config_dict

        # save_logs(log_dict, get_log_filepath(
        #     config_dict['task'], TTYPE_FORMAT, config_dict['data_code'], config_dict['exp_index']))


        # save with each indexed
        main_path = 'X:/Data Files/Huawei/'
        filename = os.path.join(main_path, config_dict['data_code'])
        model_report, model_data = model_save.create_dirs(filename, time_stamp)

        model_save_path = os.path.join(model_data, "Ensem_model---{:04d}.pt".format(cepoch))
        torch.save(ensemble_model.state_dict(), model_save_path)

        log_dict = {}
        log_dict['batch_log_list'] = batch_log_list
        log_dict['epoch_log_dict'] = epoch_log_dict
        log_dict['config_dict'] = config_dict

        model_report_path = os.path.join(model_report, "Ensem_log---{:04d}.npy".format(cepoch))
        # model_report_path = "{}.npy".format(model_report)
        np.save(model_report_path, log_dict)

    return batch_log_list, epoch_log_dict

In [25]:
loadPath = r"X:\Data Files\Huawei\mnist\exp_20221101-230528\reports\model\model---0015.pt"
config_dict = {}
config_dict['batch_size'] = 128
config_dict['learning_rate'] = 0.001
config_dict['n_layers'] = 5
config_dict['hidden_width'] = 128
config_dict['epochs'] = 50

config_dict['seed'] = 1234
config_dict['device'] = 'cuda'
config_dict['atype'] = 'relu'
config_dict['data_code'] = 'mnist'

config_dict['lambda_y'] = 100.
config_dict['sigma'] = 5.
config_dict['task'] = 'hsic-train'
config_dict['log_batch_interval'] = 10
config_dict['ext'] = 'pdf'

batch_log_list, epoch_log_dict = training_format(config_dict, loadPath)

# # # # # # # # # # # # # # # # # # # #
#     Format training
# # # # # # # # # # # # # # # # # # # #


Train Epoch: 1 [ 60032/60032 (100%)] Loss:2.3156 Acc:13.1935 hsic_xz:-1.0000 hsic_yz:-1.0000: 100%|███████████████| 469/469.0 [00:17<00:00, 26.86it/s]


Epoch - [0001]: Training Acc: 15.99
Epoch - [0001]: Testing  Acc: 15.65
Model files saved in:  X:/Data Files/Huawei/mnist\exp_20221101-233947
Model report files saved in :  X:/Data Files/Huawei/mnist\exp_20221101-233947\reports
Model saved in :  X:/Data Files/Huawei/mnist\exp_20221101-233947\reports\model


Train Epoch: 2 [ 60032/60032 (100%)] Loss:2.2246 Acc:19.4935 hsic_xz:-1.0000 hsic_yz:-1.0000: 100%|███████████████| 469/469.0 [00:17<00:00, 26.81it/s]


Epoch - [0002]: Training Acc: 23.09
Epoch - [0002]: Testing  Acc: 22.67
Model files saved in:  X:/Data Files/Huawei/mnist\exp_20221101-233947
Model report files saved in :  X:/Data Files/Huawei/mnist\exp_20221101-233947\reports
Model saved in :  X:/Data Files/Huawei/mnist\exp_20221101-233947\reports\model


...

Epoch - [0048]: Training Acc: 61.85
Epoch - [0048]: Testing  Acc: 63.01
Model files saved in:  X:/Data Files/Huawei/mnist\exp_20221101-233947
Model report files saved in :  X:/Data Files/Huawei/mnist\exp_20221101-233947\reports
Model saved in :  X:/Data Files/Huawei/mnist\exp_20221101-233947\reports\model


Train Epoch: 49 [ 60032/60032 (100%)] Loss:1.2669 Acc:61.8698 hsic_xz:-1.0000 hsic_yz:-1.0000: 100%|██████████████| 469/469.0 [00:17<00:00, 26.28it/s]


Epoch - [0049]: Training Acc: 61.91
Epoch - [0049]: Testing  Acc: 63.10
Model files saved in:  X:/Data Files/Huawei/mnist\exp_20221101-233947
Model report files saved in :  X:/Data Files/Huawei/mnist\exp_20221101-233947\reports
Model saved in :  X:/Data Files/Huawei/mnist\exp_20221101-233947\reports\model


Train Epoch: 50 [ 60032/60032 (100%)] Loss:1.2627 Acc:61.9514 hsic_xz:-1.0000 hsic_yz:-1.0000: 100%|██████████████| 469/469.0 [00:17<00:00, 26.65it/s]


Epoch - [0050]: Training Acc: 61.98
Epoch - [0050]: Testing  Acc: 63.19
Model files saved in:  X:/Data Files/Huawei/mnist\exp_20221101-233947
Model report files saved in :  X:/Data Files/Huawei/mnist\exp_20221101-233947\reports
Model saved in :  X:/Data Files/Huawei/mnist\exp_20221101-233947\reports\model


In [28]:
epoch_log_dict

{'train_acc': [10.988029,
  15.987585,
  23.091019,
  29.616428,
  35.00522,
  39.277386,
  42.630043,
  45.359142,
  47.50911,
  49.297043,
  50.666866,
  51.86012,
  52.901234,
  53.762993,
  54.4765,
  55.233322,
  55.77914,
  56.29831,
  56.746403,
  57.137306,
  57.492115,
  57.82749,
  58.137882,
  58.45327,
  58.69481,
  58.921356,
  59.178436,
  59.410534,
  59.655964,
  59.812546,
  59.970795,
  60.19123,
  60.37002,
  60.51717,
  60.654873,
  60.820343,
  60.941944,
  61.043556,
  61.124065,
  61.232895,
  61.325073,
  61.352833,
  61.42391,
  61.5344,
  61.58993,
  61.6871,
  61.700428,
  61.768723,
  61.850346,
  61.90865,
  61.98083],
 'train_loss': [2.371275,
  2.266331,
  2.184241,
  2.111375,
  2.0452833,
  1.9854494,
  1.9311193,
  1.8817402,
  1.836767,
  1.7957938,
  1.7582438,
  1.7238878,
  1.6921854,
  1.6630177,
  1.6361719,
  1.6112105,
  1.5881052,
  1.5665869,
  1.5465444,
  1.5279489,
  1.5105095,
  1.494119,
  1.4788007,
  1.4643912,
  1.4508998,
  1.4380664

# CIFAR10

In [40]:
config_dict = {}
config_dict['batch_size'] = 128
config_dict['learning_rate'] = 0.001
config_dict['n_layers'] = 5
config_dict['hidden_width'] = 128
config_dict['epochs'] = 15

config_dict['seed'] = 1234
config_dict['device'] = 'cuda'
config_dict['atype'] = 'relu'
config_dict['data_code'] = 'cifar10'

config_dict['lambda_y'] = 100.
config_dict['sigma'] = 5.
config_dict['task'] = 'hsic-train'
config_dict['log_batch_interval'] = 10
config_dict['ext'] = 'pdf'

In [41]:
batch_log_list, epoch_log_dict = training_hsic(config_dict)

# # # # # # # # # # # # # # # # # # # #
#     HSIC-Bottleneck training
# # # # # # # # # # # # # # # # # # # #
Files already downloaded and verified
Files already downloaded and verified


Train Epoch: 1 [ 50048/50048 (100%)] H_hx:0.00000042 H_hy:0.00000045: 100%|█████████| 391/391.0 [01:42<00:00,  3.83it/s]


Model files saved in:  X:/Data Files/Huawei/cifar10\exp_20221102-160343
Model report files saved in :  X:/Data Files/Huawei/cifar10\exp_20221102-160343\reports
Model saved in :  X:/Data Files/Huawei/cifar10\exp_20221102-160343\reports\model


Train Epoch: 2 [ 50048/50048 (100%)] H_hx:0.00000042 H_hy:0.00000045: 100%|█████████| 391/391.0 [01:39<00:00,  3.92it/s]


Model files saved in:  X:/Data Files/Huawei/cifar10\exp_20221102-160343
Model report files saved in :  X:/Data Files/Huawei/cifar10\exp_20221102-160343\reports
Model saved in :  X:/Data Files/Huawei/cifar10\exp_20221102-160343\reports\model


...

Model files saved in:  X:/Data Files/Huawei/cifar10\exp_20221102-160343
Model report files saved in :  X:/Data Files/Huawei/cifar10\exp_20221102-160343\reports
Model saved in :  X:/Data Files/Huawei/cifar10\exp_20221102-160343\reports\model


Train Epoch: 14 [ 50048/50048 (100%)] H_hx:0.00000042 H_hy:0.00000046: 100%|████████| 391/391.0 [01:43<00:00,  3.76it/s]


Model files saved in:  X:/Data Files/Huawei/cifar10\exp_20221102-160343
Model report files saved in :  X:/Data Files/Huawei/cifar10\exp_20221102-160343\reports
Model saved in :  X:/Data Files/Huawei/cifar10\exp_20221102-160343\reports\model


Train Epoch: 15 [ 50048/50048 (100%)] H_hx:0.00000043 H_hy:0.00000046: 100%|████████| 391/391.0 [01:43<00:00,  3.76it/s]

Model files saved in:  X:/Data Files/Huawei/cifar10\exp_20221102-160343
Model report files saved in :  X:/Data Files/Huawei/cifar10\exp_20221102-160343\reports
Model saved in :  X:/Data Files/Huawei/cifar10\exp_20221102-160343\reports\model


In [43]:
loadPath = r"X:\Data Files\Huawei\cifar10\exp_20221102-160343\reports\model\model---0015.pt"
config_dict = {}
config_dict['batch_size'] = 128
config_dict['learning_rate'] = 0.001
config_dict['n_layers'] = 5
config_dict['hidden_width'] = 128
config_dict['epochs'] = 50

config_dict['seed'] = 1234
config_dict['device'] = 'cuda'
config_dict['atype'] = 'relu'
config_dict['data_code'] = 'cifar10'

config_dict['lambda_y'] = 100.
config_dict['sigma'] = 5.
config_dict['task'] = 'hsic-train'
config_dict['log_batch_interval'] = 10
config_dict['ext'] = 'pdf'

batch_log_list, epoch_log_dict = training_format(config_dict, loadPath)

# exp_20221102-163322 23 percent

# # # # # # # # # # # # # # # # # # # #
#     Format training
# # # # # # # # # # # # # # # # # # # #
Files already downloaded and verified
Files already downloaded and verified


Train Epoch: 1 [ 50048/50048 (100%)] Loss:2.3434 Acc:11.2836 hsic_xz:-1.0000 hsic_yz:-1.0000: 100%|███████████████| 391/391.0 [00:19<00:00, 19.94it/s]


Epoch - [0001]: Training Acc: 11.62
Epoch - [0001]: Testing  Acc: 12.05
Model files saved in:  X:/Data Files/Huawei/cifar10\exp_20221102-224915
Model report files saved in :  X:/Data Files/Huawei/cifar10\exp_20221102-224915\reports
Model saved in :  X:/Data Files/Huawei/cifar10\exp_20221102-224915\reports\model


Train Epoch: 2 [ 50048/50048 (100%)] Loss:2.3166 Acc:11.9817 hsic_xz:-1.0000 hsic_yz:-1.0000: 100%|███████████████| 391/391.0 [00:19<00:00, 19.94it/s]


Epoch - [0002]: Training Acc: 12.30
Epoch - [0002]: Testing  Acc: 12.69
Model files saved in:  X:/Data Files/Huawei/cifar10\exp_20221102-224915
Model report files saved in :  X:/Data Files/Huawei/cifar10\exp_20221102-224915\reports
Model saved in :  X:/Data Files/Huawei/cifar10\exp_20221102-224915\reports\model


...

Epoch - [0048]: Training Acc: 22.77
Epoch - [0048]: Testing  Acc: 21.99
Model files saved in:  X:/Data Files/Huawei/cifar10\exp_20221102-224915
Model report files saved in :  X:/Data Files/Huawei/cifar10\exp_20221102-224915\reports
Model saved in :  X:/Data Files/Huawei/cifar10\exp_20221102-224915\reports\model


Train Epoch: 49 [ 50048/50048 (100%)] Loss:2.1172 Acc:22.8233 hsic_xz:-1.0000 hsic_yz:-1.0000: 100%|██████████████| 391/391.0 [00:19<00:00, 19.93it/s]


Epoch - [0049]: Training Acc: 22.85
Epoch - [0049]: Testing  Acc: 22.02
Model files saved in:  X:/Data Files/Huawei/cifar10\exp_20221102-224915
Model report files saved in :  X:/Data Files/Huawei/cifar10\exp_20221102-224915\reports
Model saved in :  X:/Data Files/Huawei/cifar10\exp_20221102-224915\reports\model


Train Epoch: 50 [ 50048/50048 (100%)] Loss:2.1158 Acc:22.8788 hsic_xz:-1.0000 hsic_yz:-1.0000: 100%|██████████████| 391/391.0 [00:19<00:00, 19.83it/s]


Epoch - [0050]: Training Acc: 22.90
Epoch - [0050]: Testing  Acc: 22.10
Model files saved in:  X:/Data Files/Huawei/cifar10\exp_20221102-224915
Model report files saved in :  X:/Data Files/Huawei/cifar10\exp_20221102-224915\reports
Model saved in :  X:/Data Files/Huawei/cifar10\exp_20221102-224915\reports\model
